<a href="https://colab.research.google.com/github/sarthakvarora/Binary-Tree/blob/main/Embernet_Colab_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
!mamba install -q gdal=3.4.1

warning  libmamba Could not parse mod/etag header
warning  libmamba Could not parse mod/etag header


In [4]:
from osgeo import gdal
gdal.VersionInfo()

'3040100'

In [5]:
!gdalinfo --version

GDAL 3.4.1, released 2021/12/27


In [1]:
!pip install requests
!pip install geojson
!pip install folium
!pip install geopandas
!pip install psycopg2
!pip install itertools
!pip install psycopg2
!pip install rasterio
!pip install osr
!pip install rasterstats
!pip install ogr
!pip install geoalchemy2
!pip install uuid

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools
  Using cached OSR-0.0.1.tar.gz (4.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached pyglet-1.1.4-py3-none-any.whl
  Using cached eyed3-0.9.7-py3-none-any.whl.metadata (6.0 kB)
  Using cached coverage-5.5-cp310-cp310-manylinux1_x86_64.whl.metadata (8.0 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
Using cached eyed3-0.9.7-py3-none-any.whl (246 kB)
Using cached coverage-5.5-cp310-cp310-manylinux1_x86_64.whl (238 kB)
Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Created wheel for osr: filename=OSR-0.0.1-py3-none-any.whl size=5614 sha256=3a6fb5be

In [6]:
import geopandas
import geopandas as gpd
import requests
import geojson
import folium

import psycopg2
import psycopg2.extras
import numpy as np
import itertools
import pandas as pd
import rasterio
import rasterio.mask
from osgeo import gdal
import math
import geoalchemy2
import ogr
# import osr
import uuid
import os
# import rasterstats
from collections import Counter
from joblib import Parallel, delayed
from scipy.stats import pareto
from shapely.ops import cascaded_union
from sqlalchemy import create_engine
from rasterstats import zonal_stats
from shapely import wkt
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
from shapely.wkt import loads
from sqlalchemy import text
from google.colab import drive

In [7]:
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [8]:
#Prepare the pareto distribution data

vegetation_dt = geopandas.read_file('/content/drive/MyDrive/embernet/harmonized_csv/vegetation_I.csv')
structure_dt = geopandas.read_file('/content/drive/MyDrive/embernet/harmonized_csv/components.csv')
assembly_dt = geopandas.read_file('/content/drive/MyDrive/embernet/harmonized_csv/assembly.csv')


In [9]:
ignition_dt = geopandas.read_file('/content/drive/MyDrive/embernet/IBHS_data/ignition_data.csv')

In [10]:

veg_pareto_dt = pd.DataFrame(columns=['wind_speed', 'subs','vec_len', 'b','ploc','scale'])
for j, k in itertools.product(['Idle','Medium','High'],['Grass','Shrub','Tree']):
    pareto_vec = vegetation_dt[(vegetation_dt['vegetation_type']==k) & (vegetation_dt['wind_speed'] ==j)]['total_flying_distance'].values
    # print(pareto_vec)
    pareto_vec = [float(i) for i in pareto_vec]
    b_samp, loc_samp, scale_samp = pareto.fit(pareto_vec)
    n_emb = len(pareto_vec)
    out_dt =[j,k,n_emb,b_samp,loc_samp,scale_samp]
    # veg_pareto_dt = veg_pareto_dt.append(out_dt)
    veg_pareto_dt = pd.concat([veg_pareto_dt,pd.DataFrame(columns=veg_pareto_dt.columns,data=(out_dt,),)])


# # print(vegetation_dt)
str_pareto_dt = pd.DataFrame(columns=['wind_speed','assembly','material','vec_len', 'b','ploc','scale'])

str_simple_dt = structure_dt[['wind_speed','assembly','material']].drop_duplicates()
# print()
for i in range(len(str_simple_dt)):
    samp_dt = str_simple_dt.iloc[[i]]
    # print(samp_dt)
    j=samp_dt['wind_speed'].values[0]
    k=samp_dt['assembly'].values[0]
    l=samp_dt['material'].values[0]
    # print([j,k,l])
    pareto_dt = structure_dt[(structure_dt['wind_speed']==j) &(structure_dt['assembly']==k) & (structure_dt['material']==l)]
    pareto_vec = pareto_dt['total_flying_distance'].values
    pareto_vec = [float(m) for m in pareto_vec]
    n_emb = len(pareto_vec)
    b_samp, loc_samp, scale_samp = pareto.fit(pareto_vec)
    out_dt =[j,k,l,n_emb,b_samp,loc_samp,scale_samp]
    # veg_pareto_dt = veg_pareto_dt.append(out_dt)
    str_pareto_dt = pd.concat([str_pareto_dt,pd.DataFrame(columns=str_pareto_dt.columns,data=(out_dt,),)])


asm_pareto_dt = pd.DataFrame(columns=['wind_speed','cladding','wall_type','material','roof','vec_len', 'b','ploc','scale'])

asm_simple_dt = assembly_dt[['wind_speed','cladding','wall_type','material','roof']].drop_duplicates()
# cladding 	wall_type 	material 	roof
for i in range(len(str_simple_dt)):
    samp_dt = asm_simple_dt.iloc[[i]]
    j=samp_dt['wind_speed'].values[0]
    k=samp_dt['cladding'].values[0]
    l=samp_dt['wall_type'].values[0]
    m=samp_dt['material'].values[0]
    n=samp_dt['roof'].values[0]
    # print([j,k,l])
    pareto_dt = assembly_dt[(assembly_dt['wind_speed']==j) &(assembly_dt['cladding']==k) & (assembly_dt['wall_type']==l)& (assembly_dt['material']==m)& (assembly_dt['roof']==n)]
    pareto_vec = pareto_dt['total_flying_distance'].values
    # print(pareto_vec)
    pareto_vec = [float(o) for o in pareto_vec]

    b_samp, loc_samp, scale_samp = pareto.fit(pareto_vec)
    n_emb = len(pareto_vec)
    out_dt =[j,k,l,m,n,n_emb,b_samp,loc_samp,scale_samp]
    # veg_pareto_dt = veg_pareto_dt.append(out_dt)
    asm_pareto_dt = pd.concat([asm_pareto_dt,pd.DataFrame(columns=asm_pareto_dt.columns,data=(out_dt,),)])

<ipython-input-10-1c0225e0f5cb>:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  veg_pareto_dt = pd.concat([veg_pareto_dt,pd.DataFrame(columns=veg_pareto_dt.columns,data=(out_dt,),)])
<ipython-input-10-1c0225e0f5cb>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  str_pareto_dt = pd.concat([str_pareto_dt,pd.DataFrame(columns=str_pareto_dt.columns,data=(out_dt,),)])
/usr/local/lib/python3.10/dist-packages/scipy/stats/_continuous_distns.py:7866: RuntimeWarning: divide by zero encountered in scal

In [11]:
def get_parcel_geoms(parno,county = 'Sonoma'):
    ## Set up access controlls
    #Todo: should this be some kind of 'secret' or be stored in a secrets thing?
    #Todo: check database to see if a parcel has been queried recently or if we have current enough data
    # and returned that analysis instead of buying another parcel and analyisin
    db_cred = {'user': 'jifesypi',
                'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
                'host': 'mild-williams-pear.db.elephantsql.com',
                'port': '5432',
                'database': 'jifesypi',
                'epsg': '4326',
                'schema':'schema'
                }

    #Create connection to the database
    engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

    sql_query_parcels = f"SELECT * FROM public.california_parcels WHERE parno = '{parno}' AND county = '{county}'"

    parcel_dataframe = geopandas.read_postgis(sql_query_parcels, engine)

    #Set parcel number
    # parcel_dataframe= sonoma_parcel_dataframe[sonoma_parcel_dataframe['parno'] == parno]


    print(parcel_dataframe)
    # Query postgres database based on parcel geometry
    #Set query EPSG (WSG 4326)
    parcel_dataframe = parcel_dataframe.set_crs('EPSG:4326')

    #Convert to 26910, stateplane & meters
    parcel_dataframe_26910 = parcel_dataframe.to_crs(26910)

    #Buffer parcel 1000 meters
    buffer_parcel_dataframe_26910 = parcel_dataframe_26910.buffer(1000).to_crs(4326)

    #Extract well known text
    buffer_parcel_geometry_wkt = buffer_parcel_dataframe_26910.geometry.to_wkt().to_list()[0]
    parcel_geometry_wkt = parcel_dataframe_26910.to_crs(4326).geometry.to_wkt().to_list()[0]

    #Query for neighborhood of parcels
    sql_query_parcels = f"SELECT * FROM public.california_parcels WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{buffer_parcel_geometry_wkt}', 0))"

    #Send to engine
    neighborhood_parcels = geopandas.read_postgis(sql_query_parcels, engine)


    #Need to union to send to wkt query
    # neighborhood_parcels_union = geopandas.GeoSeries(unary_union(neighborhood_parcels.geometry.values))
    neighborhood_parcels_union = neighborhood_parcels.unary_union

    #Get wkt of neighborhood of parcels
    neighborhood_geometry_wkt = neighborhood_parcels_union.wkt

    #Query for extended neighborhood (1000m) of structures
    sql_query_parcels = f"SELECT * FROM public.california_structures WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{neighborhood_geometry_wkt}',0))"

    neighborhood_structures = geopandas.read_postgis(sql_query_parcels, engine)

    #Query for parcel structures
    sql_query_structures = f"SELECT * FROM public.california_structures WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{parcel_geometry_wkt}',0))"

    parcel_structures = geopandas.read_postgis(sql_query_structures, engine)
    engine.dispose()
    return parcel_dataframe, neighborhood_parcels, parcel_structures, neighborhood_structures

In [12]:
def update_vegetation_neighborhood(neighborhood_parcels,county = 'Sonoma', do_all = True, update=True ,return_gpd = False):
    filename = str(uuid.uuid4())

    db_cred = {'user': 'jifesypi',
            'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
            'host': 'mild-williams-pear.db.elephantsql.com',
            'port': '5432',
            'database': 'jifesypi',
            'epsg': '4326',
            'schema':'schema'
            }

    #Create connection to the database
    engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

    parno_values = ','.join([f"'{parno}'" for parno in neighborhood_parcels['parno']])

    # print(parno_values)

    # Construct the SQL query
    sql_query_parcels = f"SELECT * FROM public.parcel_veg WHERE parno IN ({parno_values}) AND county = '{county}'"

    previously_completed = geopandas.read_postgis(sql_query_parcels, engine)

    if do_all== False:
        neighborhood_parcels = neighborhood_parcels[~neighborhood_parcels['parno'].isin(previously_completed['parno'])]

    if neighborhood_parcels.shape[0] >0:
        # Need to union to send to wkt query
        neighborhood_parcels_union = neighborhood_parcels.unary_union

        #Get wkt of neighborhood of parcels
        neighborhood_geometry_wkt = neighborhood_parcels_union.wkt

        #Query for extended neighborhood (1000m) of structures
        sql_query_parcels = f"SELECT * FROM public.veg_map WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{neighborhood_geometry_wkt}',0))"

        neighborhood_veg = geopandas.read_postgis(sql_query_parcels, engine)
        engine.dispose()

        with rasterio.open("/content/Stack_Sonoma_2021_biomass.tif") as src:
            biomass_out_image, biomass_out_transform = rasterio.mask.mask(src, neighborhood_parcels.geom, crop=True)
            biomass_out_meta = src.meta

        biomass_out_meta.update({"driver": "GTiff",
                        "height": biomass_out_image.shape[1],
                        "width": biomass_out_image.shape[2],
                        "transform": biomass_out_transform})

        with rasterio.open("/content/processing_folder/"+filename+"_biomass.tif", "w", **biomass_out_meta) as dest:
            dest.write(biomass_out_image)
        # from osgeo import gdal, ogr, osr

        in_path = '/content/processing_folder/'+filename+'_class.tif'

        out_path = '/content/processing_folder/'+filename+'_class.gpkg'

        vegetation_dataframe = gpd.overlay(neighborhood_parcels, neighborhood_veg, how='intersection')

        vegetation_dataframe[['Class_str']] =  vegetation_dataframe[['LF_FOREST']].values
        d = {
            'Agriculture' : 'Grass' ,
            'Aquatic Vegetation' : 'Shrub',
            'Barren and Sparsely Vegetated' : 'Grass',
            'Conifer Forest' : 'Tree',
            'Developed' : 'Grass',
            'Forest Sliver' : 'Tree',
            'Hardwood Forest' : 'Tree',
            'Herbaceous': 'Grass',
            'Herbaceous Wetland' : 'Tree',
            'Mixed Conifer-Hardwood Forest' : 'Tree',
            'Non-native Forest' : 'Tree',
            'Non-native Shrub' : 'Shrub',
            'Riparian Forest' : 'Tree',
            'Riparian Shrub' : 'Shrub',
            'Salt Marsh' : 'Grass',
            'Shrub' : 'Shrub',
            'Water' : 'Grass'}
        vegetation_dataframe.Class_str = vegetation_dataframe.Class_str.map(d)

        vegetation_dataframe['biomass'] = zonal_stats(vegetation_dataframe,
                                "/content/processing_folder/"+filename+"_biomass.tif",
                                stats="mean", # any other stat will also do
                                categorical=False,
                                nodata = np.nan)
        vegetation_dataframe['biomass'] = [i['mean'] for i in vegetation_dataframe['biomass']]
        vegetation_dataframe['area'] = vegetation_dataframe.to_crs(6414).area/10000
        vegetation_dataframe['MG'] = vegetation_dataframe['area']*(vegetation_dataframe['biomass'])
        emb_vec = []
        print(vegetation_dataframe)

        # vegetation_dataframe['Class_str'] = np.nan_to_num(vegetation_dataframe['Class_str'], nan='Grass')
        vegetation_dataframe['Class_str'] = vegetation_dataframe['Class_str'].fillna('Grass')
        print(vegetation_dataframe)
        for i in range(len(vegetation_dataframe)):
            iloc_dt = vegetation_dataframe.iloc[[i]]
            print(iloc_dt['Class_str'].values[0])
            emb_vec.append(veg_pareto_dt[(veg_pareto_dt.subs == iloc_dt['Class_str'].values[0] ) &(veg_pareto_dt.Wind_Speed == 'High')].vec_len.values[0])
        vegetation_dataframe['embers_base'] = emb_vec
        #The division by five is for the assumed kg of vegetation used in the development of the distributions
        #The multiplication is kg -> MG
        vegetation_dataframe['embers'] = vegetation_dataframe['MG']*vegetation_dataframe['embers_base']*1000/5
        vegetation_dataframe = vegetation_dataframe.fillna(0)
        os.remove("/content/processing_folder/"+filename+"_biomass.tif")


        if update:
            # Start a transaction
            with engine.begin() as conn:
                # Remove rows with existing 'parno' values
                existing_parnos = vegetation_dataframe['parno'].unique()
                # print(existing_parnos)
                existing_parnos_str = ','.join(f"'{parno}'" for parno in existing_parnos) # surround each parno with quotes and join them with commas
                print(existing_parnos_str)
                delete_query = f"DELETE FROM parcel_veg WHERE parno IN ({existing_parnos_str})"
                conn.execute(delete_query)



            vegetation_dataframe["geom"] = vegetation_dataframe["geometry"]

            vegetation_dataframe["geom"] = [MultiPolygon([feature]) if isinstance(feature, Polygon) else feature for feature in vegetation_dataframe["geom"]]

            vegetation_dataframe.set_geometry("geom")
            vegetation_dataframe = vegetation_dataframe.drop('geometry', axis=1)

            # zones = zones.loc[zones.geometry.geometry.type=='MultiPolygon']
            vegetation_dataframe['geom'] = vegetation_dataframe['geom'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, srid=4326))

            vegetation_dataframe.to_sql('parcel_veg', engine, if_exists='append', index=False,
                            dtype={'geom': geoalchemy2.Geometry('MultiPolygon', srid= 4326)})
            engine.dispose()

    if return_gpd:
        #Query for parcel structures
        sql_query_parcels = f"SELECT * FROM public.parcel_veg WHERE parno IN ({parno_values}) AND county = '{county}'"
        vegetation_dataframe = geopandas.read_postgis(sql_query_parcels, engine)
        engine.dispose()
        return vegetation_dataframe

In [13]:
def get_structure_pareto(parcel_structures):
    n_embers = str_pareto_dt[(str_pareto_dt.assembly == 'C' )&(str_pareto_dt.material == 'CO' ) &(str_pareto_dt.wind_speed == 'High')].vec_len[0]
    parcel_structures['embers'] = n_embers*parcel_structures.to_crs(6414).area*6
    return(parcel_structures)

In [14]:
def structure_zone_gen(i_samp,structure_dataframe):
    iloc_new_dt = structure_dataframe.iloc[[i_samp]]
    b_ = str_pareto_dt[(str_pareto_dt.assembly == 'C' )&(str_pareto_dt.material == 'CO' ) &(str_pareto_dt.wind_speed == 'High')].b.values[0]
    loc_ = str_pareto_dt[(str_pareto_dt.assembly == 'C' )&(str_pareto_dt.material == 'CO' ) &(str_pareto_dt.wind_speed == 'High')].ploc.values[0]
    scale_ = str_pareto_dt[(str_pareto_dt.assembly == 'C' )&(str_pareto_dt.material == 'CO' ) &(str_pareto_dt.wind_speed == 'High')].scale.values[0]

    emb_count =  int(max(iloc_new_dt['embers'].values[0],75000))

    scale = 1

    if emb_count > 1000000:
        scale = emb_count/1000000
        emb_count = 1000000

    emb_dist = pareto.rvs(size=emb_count,b=b_,loc=loc_,scale=scale_)

    emb_dist = [int(math.ceil(i)) for i in emb_dist]


    values, counts = np.unique(emb_dist, return_counts=True)

    counts =  counts*scale

    obj_r = math.sqrt(iloc_new_dt.to_crs(6414).area/3.14)

    values_area = (3.14*(obj_r+values)*(obj_r+values)) - (3.14*(obj_r+values -1)*(obj_r+values -1))

    counts_area = counts/values_area

    max_v_vec = values <= max(values[counts_area > min(1,max(counts_area-1))])

    values = values[max_v_vec]

    counts = counts[max_v_vec]

    counts_area = counts_area[max_v_vec]

    #This is already a short cut. I'd rather not short cut any more than k == 15.
    k = 30
    l = (len(counts)-1)
    k_vec =  np.sort([i for i in range(min(5,len(counts)))]+[l]+[np.ceil(l*(1/(i+1))) for i in range(k)][::-1])
    k_vec = np.sort(list(set(k_vec)))
    k_vec = [int(i) for i in k_vec]
    norm_count_vec = (np.cumsum(counts)[k_vec])/np.sum(counts)
    norm_0_count_vec = np.insert(norm_count_vec, 0, 0)[0:len(norm_count_vec)]
    counts_vec = np.sum(counts)*(norm_count_vec-norm_0_count_vec)
    values_vec = values[k_vec]

    counts, values = counts_vec,values_vec

    xval_low = np.insert(values, 0, 0)[0:(len(values))]
    xval_high = np.insert(values, 0, 0)[1:(len(values)+1)]

    # print(xval_low)
    # print(xval_high)
    def zone_out(xval_low,xval_high):
        zone_low = iloc_new_dt.geometry.buffer(xval_low).unary_union
        zone_high = iloc_new_dt.geometry.buffer(xval_high).unary_union
        return zone_high.difference(zone_low)
    iloc_new_dt = iloc_new_dt.to_crs(6414)

    zone_vec = np.vectorize(zone_out)(xval_low,xval_high)

    zone_vec = np.append(iloc_new_dt.geometry.to_crs(6414),zone_vec)
    counts = np.append(counts[0],counts)
    values = np.append(values[0],values)

    zones = geopandas.GeoDataFrame()
    # zones.is_copy = False

    zones['counts'] = counts
    zones['values'] = values
    zones['values'][0] = 0
    zones.geometry = geopandas.GeoSeries(zone_vec)
    zones['area_r'] =  zones.geometry.area
    counts_area_r_vec = zones['counts']/zones['area_r']
    counts_area_r_vec[0] = counts_area_r_vec[1]
    zones['counts_area_r'] = counts_area_r_vec
    # zones['counts_area_r'][0] = zones['counts_area_r'][1]
    # zones['class'] = np.repeat(iloc_new_dt['Class_str'][0],len(zones['counts_area_r']))
    zones['Class_str'] = 'Structure'
    zones['Class'] = 15
    iloc_new_dt['area'] = iloc_new_dt.area

    zones['area'] =  iloc_new_dt['area'].iloc[0]

    zones = zones.set_crs('EPSG:6414')
    zones = zones.to_crs(4326)
    return zones

In [15]:
def veg_zone_gen(i_samp,vegetation_dataframe):
    iloc_new_dt = vegetation_dataframe.iloc[[i_samp]]
    b_ = veg_pareto_dt[(veg_pareto_dt.subs == iloc_new_dt['Class_str'].values[0] ) &(veg_pareto_dt.wind_speed == 'High')].b.values[0]
    loc_ = veg_pareto_dt[(veg_pareto_dt.subs == iloc_new_dt['Class_str'].values[0] ) &(veg_pareto_dt.wind_speed == 'High')].ploc.values[0]
    scale_ = veg_pareto_dt[(veg_pareto_dt.subs == iloc_new_dt['Class_str'].values[0] ) &(veg_pareto_dt.wind_speed == 'High')].scale.values[0]

    emb_count =  int(max(iloc_new_dt['embers'].values[0],75000))

    scale = 1

    if emb_count > 1000000:
        scale = emb_count/1000000
        emb_count = 1000000

    emb_dist = pareto.rvs(size=emb_count,b=b_,loc=loc_,scale=scale_)

    emb_dist = [int(math.ceil(i)) for i in emb_dist]


    values, counts = np.unique(emb_dist, return_counts=True)

    counts =  counts*scale

    obj_r = math.sqrt(iloc_new_dt.to_crs(6414).area/3.14)

    values_area = (3.14*(obj_r+values)*(obj_r+values)) - (3.14*(obj_r+values -1)*(obj_r+values -1))

    counts_area = counts/values_area

    max_v_vec = values <= max(values[counts_area > min(1,max(counts_area-1))])

    values = values[max_v_vec]

    counts = counts[max_v_vec]

    counts_area = counts_area[max_v_vec]

    #This is already a short cut. I'd rather not short cut any more than k == 15.
    k = 30
    l = (len(counts)-1)
    k_vec =  np.sort([i for i in range(min(5,len(counts)))]+[l]+[np.ceil(l*(1/(i+1))) for i in range(k)][::-1])
    k_vec = np.sort(list(set(k_vec)))
    k_vec = [int(i) for i in k_vec]
    norm_count_vec = (np.cumsum(counts)[k_vec])/np.sum(counts)
    norm_0_count_vec = np.insert(norm_count_vec, 0, 0)[0:len(norm_count_vec)]
    counts_vec = np.sum(counts)*(norm_count_vec-norm_0_count_vec)
    values_vec = values[k_vec]

    counts, values = counts_vec,values_vec

    xval_low = np.insert(values, 0, 0)[0:(len(values))]
    xval_high = np.insert(values, 0, 0)[1:(len(values)+1)]

    # print(xval_low)
    # print(xval_high)
    def zone_out(xval_low,xval_high):
        zone_low = iloc_new_dt.geometry.buffer(xval_low).unary_union
        zone_high = iloc_new_dt.geometry.buffer(xval_high).unary_union
        return zone_high.difference(zone_low)
    iloc_new_dt = iloc_new_dt.to_crs(6414)
    zone_vec = np.vectorize(zone_out)(xval_low,xval_high)

    zone_vec = np.append(iloc_new_dt.geometry.to_crs(6414),zone_vec)
    counts = np.append(counts[0],counts)
    values = np.append(values[0],values)

    zones = geopandas.GeoDataFrame()
    # zones.is_copy = False

    zones['counts'] = counts
    zones['values'] = values
    zones['values'][0] = 0
    # print(values)
    # print(xval_low)
    # zones['low'] = xval_low[0:(len(values))]
    # zones['high'] = xval_high[0:(len(values))]


    zones.geometry = geopandas.GeoSeries(zone_vec)
    zones['area_r'] =  zones.geometry.area
    counts_area_r_vec = zones['counts']/zones['area_r']
    counts_area_r_vec[0] = counts_area_r_vec[1]
    zones['counts_area_r'] = counts_area_r_vec
    # zones['counts_area_r'][0] = zones['counts_area_r'][1]
    # zones['class'] = np.repeat(iloc_new_dt['Class_str'][0],len(zones['counts_area_r']))

    iloc_new_dt['area'] = iloc_new_dt.area
    zones['area'] = iloc_new_dt['area'].iloc[0]
    zones['Class_str'] = iloc_new_dt['Class_str'].iloc[0]
    zones['LF_FOREST'] = iloc_new_dt['LF_FOREST'].iloc[0]
    zones['MG'] = iloc_new_dt['MG'].iloc[0]

    zones = zones.set_crs('EPSG:6414')
    zones = zones.to_crs(4326)
    return zones

In [16]:
def build_network_kpartite(PARNO = '182-390-032', do_all = True, update=True ,return_gpd = False):
    db_cred = {'user': 'jifesypi',
            'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
            'host': 'mild-williams-pear.db.elephantsql.com',
            'port': '5432',
            'database': 'jifesypi',
            'epsg': '4326',
            'schema':'schema'
            }
    engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")
    # sql_query_parcels = f"SELECT * FROM public.embernet_kpartite WHERE parno_1 = '{PARNO}'"
    # sample_summary = geopandas.read_postgis(sql_query_parcels, engine)
    # engine.dispose()
    # # sample_summary = []
    # does_not_exist = len(sample_summary) == 0
    # print(does_not_exist)
    # if len(sample_summary) > 0:
    #     sample_summary['geom'] = sample_summary['geom'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, srid=4326))

    if (do_all or does_not_exist):
        parcel_dataframe, Null, parcel_structures, Null = get_parcel_geoms(PARNO)

        parcel_veg = update_vegetation_neighborhood(parcel_dataframe,update=True,return_gpd =True,do_all=False)
        parcel_veg['area'] = parcel_veg.area

        veg_zone_gen_vec = []
        if(len(parcel_dataframe)>0):
            neighborhood_veg= parcel_veg[[not value for value in np.isin(parcel_veg['LF_FOREST'] ,  ['Developed','Water'])]]
            veg_zone_gen_vec = [veg_zone_gen(i_samp=i_samp,vegetation_dataframe= parcel_veg) for i_samp in range(len(parcel_veg))]

        structure_zone_gen_vec = []
        if(len(parcel_structures)>0):
            structure_dataframe = get_structure_pareto(parcel_structures)
            structure_zone_gen_vec = [structure_zone_gen(m,structure_dataframe) for m in range(len(structure_dataframe))]

        if((len(structure_zone_gen_vec)> 0) & (len(veg_zone_gen_vec)> 0) ):
            zone_vec = [pd.concat(structure_zone_gen_vec),pd.concat(veg_zone_gen_vec)]

        if((len(structure_zone_gen_vec)== 0) & (len(veg_zone_gen_vec)> 0) ):
            zone_vec = [pd.concat(veg_zone_gen_vec)]

        if((len(structure_zone_gen_vec)> 0) & (len(veg_zone_gen_vec)== 0) ):
            zone_vec = [pd.concat(structure_zone_gen_vec)]

        if((len(structure_zone_gen_vec)== 0) & (len(veg_zone_gen_vec)== 0) ):
            zone_vec = []

        zones = pd.concat(zone_vec)


        bounding_box = gpd.GeoDataFrame(geometry=[zones.geometry.unary_union.envelope])

        #Get wkt of neighborhood of parcels
        neighborhood_geometry_wkt = bounding_box['geometry'].to_wkt()[0]

        #Query for extended neighborhood (1000m) of structures

        engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

        sql_query_parcels = f"SELECT * FROM public.parcel_veg WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{neighborhood_geometry_wkt}',0))"

        neighborhood_veg = geopandas.read_postgis(sql_query_parcels, engine)

        sql_query_parcels = f"SELECT * FROM public.california_parcels WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{neighborhood_geometry_wkt}',0))"

        region_parcels = geopandas.read_postgis(sql_query_parcels, engine)

        sql_query_structures = f"SELECT * FROM public.california_structures WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{neighborhood_geometry_wkt}',0))"

        region_structures = geopandas.read_postgis(sql_query_structures, engine)

        result = [not value for value in np.isin(region_parcels['parno'], neighborhood_veg['parno'])]


        neighborhood_veg = update_vegetation_neighborhood(region_parcels,update=True,return_gpd =True,do_all=False)

        zones['parno'] = PARNO

        zones = zones.to_crs(6414)
        neighborhood_veg = neighborhood_veg.to_crs(6414)
        region_structures = region_structures.to_crs(6414)
        region_parcels = region_parcels.to_crs(6414)

        # print(zones)
        # zones['area'] = zones.area
        neighborhood_veg['area'] = neighborhood_veg.area
        region_structures['area'] = region_structures.area

        neighborhood_veg_zones = geopandas.overlay(
            zones,
            neighborhood_veg,
            how="intersection")

        region_structures['Class_str'] = 'Structure'
        parno_region_structures = geopandas.overlay(
            region_structures,
            region_parcels,
            how="intersection")



        neighborhood_structure_zones = geopandas.overlay(
            zones,
            parno_region_structures,
            how="intersection")

        sample_connection = pd.concat([neighborhood_veg_zones,neighborhood_structure_zones])

        # print(sample_connection)


        sample_connection['area'] = sample_connection.area

        sample_connection['embers'] = sample_connection['counts_area_r']*sample_connection['area']

        # print(sample_connection)
        # sample_connection['dist_min'] = sample_connection['values']
        sample_connection['dist_min'] = sample_connection['values']
        sample_connection['dist_max'] = sample_connection['values']

        sample_connection['MG_2'] = sample_connection['MG_2'].fillna(0)
        sample_connection['MG_1'] = sample_connection['MG_1'].fillna(0)

        # print(sample_connection[sample_connection['Class_str_2'] == 'Structure'][['Class_str_2','Class_str_1','parno_2','parno_1','embers','area','MG_2','MG_1','area_2','area_1','dist_min','dist_max']])

        sample_summary = sample_connection.groupby(['parno_2','parno_1','Class_str_2','Class_str_1']).agg({'embers': 'sum','area':'sum','MG_2':'sum','MG_1':'sum','area_2':'mean','area_1':'mean','dist_min':'min','dist_max':'max'}).reset_index()


        sample_summary['embM2'] = sample_summary['embers'] / sample_summary['area'] # simplified the lambda function

        # Get centroids of the target parcel cover by class and parno
        parcel_cover_zones = sample_connection[['parno_2', 'Class_str_2', 'geometry']].dissolve(by=['Class_str_2', 'parno_2']).reset_index()
        parcel_cover_zones['class_parno_centroid'] = parcel_cover_zones.centroid

        # Get centroids of all parcels
        parcel_zones = neighborhood_veg_zones[['parno_2', 'geometry']].dissolve(by=['parno_2']).reset_index()
        parcel_zones['parcel_centroid'] = parcel_zones.centroid

        # Create a dictionary of target centroids with (Class_str_2, parno_2) as key
        targ_centroids = dict(zip(zip(parcel_cover_zones['Class_str_2'], parcel_cover_zones['parno_2']), parcel_cover_zones['class_parno_centroid']))

        # Assign target centroids to each class in the summary
        sample_summary['geom'] = sample_summary.apply(lambda row: targ_centroids.get((row['Class_str_2'], row['parno_2'])), axis=1)

        sample_summary['geom'] = sample_summary['geom'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, srid=6414))
        # print(sample_summary)

        # sample_summary = sample_summary.to_crs(4326)

    if update:
        # Start a transaction
        with engine.begin() as conn:
            print(conn)
            # Remove rows with existing 'parno' values
            existing_parnos = sample_summary['parno_1'].unique()
            existing_parnos_str = ','.join(f"'{parno_1}'" for parno_1 in existing_parnos) # surround each parno with quotes and join them with commas

            print(existing_parnos_str)
            delete_query = text(f"DELETE FROM embernet_kpartite WHERE parno_1 IN ({existing_parnos_str})")
            conn.execute(delete_query)

        # sample_summary['geom'] = sample_summary['geom'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, srid=4326))

        engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

        sample_summary.to_sql('embernet_kpartite', engine, if_exists='append', index=False,
                        dtype={'geom': geoalchemy2.Geometry('Point', srid= 6414)})
        engine.dispose()

    if return_gpd:
        print(PARNO)
        engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")
        sql_query_parcels = f"SELECT * FROM public.embernet_kpartite WHERE parno_1 = '{PARNO}'"

        sample_summary = geopandas.read_postgis(sql_query_parcels, engine)
        engine.dispose()

        return(sample_summary)

In [17]:
# build_network_kpartite(PARNO = '016-830-009', do_all = True, update=True, return_gpd = False)

In [18]:
import multiprocessing

num_cores = multiprocessing.cpu_count()

print(f"Number of cores: {num_cores}")

Number of cores: 2


In [20]:
import os
path = '/content/drive/MyDrive/embernet/CWOs/'
files = os.listdir(path)
print(files[2])
for file_name in [files[2]]:
    CWO_path = path+file_name
    CWO_pd = geopandas.read_file(CWO_path)
    CWO_pd = CWO_pd.dissolve()
    #Query for neighborhood of parcels
    CWO_pd_buffer = CWO_pd.to_crs(6414).buffer(3000)
    district_wkt_4326 = CWO_pd_buffer.to_crs(4326).geometry.iloc[0].wkt
    district_wkt_6414 = CWO_pd_buffer.to_crs(6414).geometry.iloc[0].wkt
    db_cred = {'user': 'jifesypi',
        'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
        'host': 'mild-williams-pear.db.elephantsql.com',
        'port': '5432',
        'database': 'jifesypi',
        'epsg': '4326',
        'schema':'schema'
        }

    #Query for neighborhood of parcels
    district_wkt = CWO_pd_buffer.geometry.iloc[0].wkt
    engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")
    sql_query_parcels = f"SELECT * FROM california_parcels WHERE ST_Intersects(california_parcels.geom, ST_GeomFromText('{district_wkt_4326}',4326));"

    #Send to engine
    community_parcels = geopandas.read_postgis(sql_query_parcels, engine)

    #Send to engine
    sql_query_parcels = f"SELECT * FROM embernet_kpartite WHERE ST_Intersects(embernet_kpartite.geom, ST_GeomFromText('{district_wkt_6414}',6414));"
    ember_net_complete = geopandas.read_postgis(sql_query_parcels, engine)

    # incomplete_list = [not item for item in community_parcels['parno'].isin(ember_net_complete['parno_1'])]

    # community_parcels = community_parcels[incomplete_list]

    # neighborhood_veg = update_vegetation_neighborhood(community_parcels,update=False,return_gpd =False,do_all=False)
    present_values = community_parcels['parno'][community_parcels['parno'].isin(ember_net_complete['parno_1'])].unique()
    missing_values = community_parcels['parno'][~community_parcels['parno'].isin(ember_net_complete['parno_1'])].unique()

    print(f"Values present in both columns are:\n{len(present_values)}")
    print(f"Values missing from second column are:\n{len(missing_values)}")
    # print(len(community_parcels['parno']))
    # for PARNO in parcel_dataframe['parno']:
    def process(i):
        try:
            build_network_kpartite(PARNO = i, do_all = True, update=True, return_gpd = False)
            result = None
        except Exception as e:
            print(f"An error occurred for input {i}: {str(e)}")
            result = None
        return result

    results = Parallel(n_jobs=16)(delayed(process)(i) for i in missing_values)

SeaRanchParcels.gpkg
Values present in both columns are:
0
Values missing from second column are:
3473


In [21]:
results

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,